# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$

__<font color='red'>Crash! B is not correct!</font>__


![Minion](https://octodex.github.com/images/dojocat.jpg)


$$
B = D^{-1} (D - A)
$$

Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [2]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [4]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [3]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [5]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

## My improvement
If we represent $A = D + \Lambda$, we can get formal solution:
$$
x = A^{-1}b = (D + \Lambda)^{-1}b = (D\cdot(I + D^{-1}\Lambda)^{-1}b = (I - D^{-1}\Lambda + ..)\cdot D^{-1}b
$$
Thus we are getting first approximation $x_1$ 
Conseqntmetly, $$x_0 = c$$


In [13]:
xmy = BB @ c + c
print('Solution', x)
print('My first iteration', xmy)
print('|x_{my} - x|',np.linalg.norm(x-xmy))
print('|x_{my} - x_1|', np.linalg.norm(xmy-(BB @ x0 + c)))

Solution [ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]
My first iteration [ 0.03584109  0.03476299  0.04014701  0.02028795  0.05563685 -0.00308751
 -0.00748446  0.02956317 -0.00719261  0.05022617]
|x_{my} - x| 0.008577261943580864
|x_{my} - x_1| 0.94981803184979


In [14]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.73472348e-17,  0.00000000e+00,
       -2.77555756e-17,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [15]:
    def jacobi_solution(A, b, n_iter = 15):
        diag_1d = np.diag(A)
        B = -A.copy()
        np.fill_diagonal(B, 0)

        D = np.diag(diag_1d)
        invD = np.diag(1./diag_1d)
        B = invD @ B 
        c = invD @ b
        
        x = c
        for _ in range(n_iter):
            x = B @ x + c
        return x

In [28]:
test = jacobi_solution(A,b,25)
print(np.linalg.norm(test-xx))

rndm = np.random.RandomState(1488)

n = 10
A_test = rndm.uniform(size=(n, n))
b_test = rndm.uniform(size=n)
x_test = jacobi_solution(A_test, b_test, 25)
print(A_test @ x_test - b_test)

2.397582587887413e-15
[-7.52378901e+29 -5.51006433e+29 -5.15519979e+29 -3.88306315e+29
 -3.23568376e+29 -4.77530047e+29 -6.92027002e+29 -6.17662403e+29
 -7.77904800e+29 -6.80397901e+29]


In [29]:
    def jacobi_solution(A, b, n_iter = 15):
        diag_1d = np.diag(A)
        B = -A.copy()
        np.fill_diagonal(B, 0)

        D = np.diag(diag_1d)
        invD = np.diag(1./diag_1d)
        B = invD @ B 
        c = invD @ b
        norm = np.linalg.norm(B)
        if norm > 1 - 1e-15:
            print('You are trying use Jacobi for non-diagol-dominant matrix, norm B = ', norm)
            a = 1/0
        x = c
        for _ in range(n_iter):
            x = B @ x + c
        return x

In [30]:

rndm = np.random.RandomState(1488)

n = 10
A_test = rndm.uniform(size=(n, n))
b_test = rndm.uniform(size=n)
x_test = jacobi_solution(A_test, b_test, 25)

You are trying use Jacobi for non-diagol-dominant matrix, norm B =  22.480213997525233


ZeroDivisionError: division by zero

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

Working formule:
$$
(D+L)x_{n+1} = b - U x_{n}
$$

In [76]:
def seidel(A, b, n_iter=15):
    d = np.diag(A)
    D = np.diag(d)
    a = A - D
    L = np.tril(a)
    U = a - L
    G = D + L
    x0 = np.diag(1./d) @ b
    x = x0
    for _ in range(n_iter):
        c = b - U @ x
        x = get_next(G, c)
        #print(x)
    return x

In [72]:
def get_next(G, c):
    x = np.zeros(G.shape[0])
    for i in range(G.shape[0]):
        x[i] = (c[i] - G[i,:] @ x) / G[i,i]
    return x

In [77]:
rndm = np.random.RandomState(1488)

n = 10
A_t = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b_t = rndm.uniform(size=n)
x_t = seidel(A_t, b_t, 10)

In [78]:
A_t @ x_t - b_t

array([-1.73028258e-13, -9.61453139e-14, -8.02691247e-14, -3.14193116e-14,
       -8.04911693e-16,  3.88578059e-15,  2.13162821e-14,  7.46624984e-15,
        9.88098492e-15,  0.00000000e+00])

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

## Constructing plan
Consider image, has non-movement point in solution our equation:
$$
\varphi(x) = x - \alpha \cdot r(x),
$$
where $r(x) = A x - b$ and $\alpha$ - some real number.
Then we are make iteration machine:
$$x_{n+1} = \varphi(x_n)$$
with dependecy $\alpha(n)$.

To minimization error $r_{n+1}$ we should to pick $$\alpha_n = \frac{(A r_n, r_n)}{(A r_n, A r_n)}$$